In [1]:
import random
import json
import pandas as pd
from faker import Faker
from faker.providers import phone_number
from datetime import datetime, timedelta

In [2]:
fake = Faker(["en_US"])

meeting_chunks = [" team meeting", " status update", " discussion", " project meeting", " seminar", " workshop"]

rooms = ["Alpha", "Beta", "Gamma", "Delta", "Epsilon", "Zeta", "Sigma", "Omega"]
people = []

group_names = ["Administration", "Biology", "Climate", "Engineering", "Healthcare", "Mathematics", "Philosophy", "Statistics"]
groups = {g: [] for g in group_names}


for i in range(10):
    name = fake.name()
    email = name.lower().replace(" ", ".") + "@woz.email"
    phone = "+47" + fake.msisdn()[:8]
    group_assignment = random.choice(group_names)
    profile = {"name": name, "index": i, "email": email, "phone": phone, "calendar": []}
    people.append(profile)
    groups[group_assignment].append(profile)
    #print(profile)

#for g, v in groups.items():
#    print(g, v)

In [3]:
class FakeOrg():
    
    def __init__(self, group_names, rooms, meeting_chunks, n_people=1):
    
        self.fake = Faker(["en_UK"])
        self.group_names = group_names
        self.rooms = rooms
        self.meeting_chunks = meeting_chunks
        
        self.people = []
        self.events = []

        self.groups = {g: [] for g in group_names}
        self.people_calendars = {p["name"]: [] for p in people}
        self.group_calendars = {g: [] for g in group_names}
        self.room_calendars = {r: [] for r in rooms}
        
        for i in range(n_people):
            self.create_person()

    def create_person(self):
        
        name = "-"
        while len(name.split()) != 2 or "-" in name:
            name = self.fake.name() # Ensure only first and last name (no e.g. "Dr John Doe, Jane Doe MD")
        email = name.lower().replace(" ", ".") + "@woz.email"
        phone = "+47" + self.fake.msisdn()[:8]
        group_assignment = random.choice(self.group_names)
        profile = {"name": name, "index": len(self.people), "email": email, "phone": phone, "group": group_assignment, "calendar": []}
           
        self.people.append(profile)
        self.people_calendars[name] = []   
        self.groups[group_assignment].append(profile)

    def create_event(self, organizer=None, n_attendees=2):

        if organizer == None:
            organizer = random.choice(self.people)
            
        full_date = self.fake.date_time_between(start_date='now', end_date='+60d') #Generate fake date
        event_date = full_date.date() # Select only the date

        event_hour = random.choice([i for i in range(8, 16)]) # Make sure the event start hour is between 8 and 4
        event_minutes = random.choice([0, 30]) # Make some events start and end on a half hour

        event_start = full_date.time().replace(hour=event_hour, minute=event_minutes, second=0) # Start time, rounded to the hour
        event_end = event_start.replace(hour=event_hour+1, minute=event_minutes, second=0) # End time, rounded to the hour

        event_id = fake.sha256()[:10]
        event_group = organizer["group"] # Assign event to the same group as the organizer
        event_location = random.choice(self.rooms) # Choose a room randomly for the meeting
        
        #event_name = random.choice([fake.bs().split()[2], event_group]) + random.choice(self.meeting_chunks) #Fake event name
        event_name = fake.bs().split()[2] + random.choice(self.meeting_chunks) #Fake event name

        #event_group = random.choice(self.group_names) # Choose a group randomly for the meeting

        group_members = self.groups[event_group]
        
        event_attendees = [organizer["name"]]
        attendee_profiles = [a for a in random.sample(group_members, k=min(n_attendees, len(self.groups[event_group]))) if not a["name"] == organizer["name"]]
        attendee_indices = {a["name"]: a["index"] for a in attendee_profiles} # To find in people list
        attendee_indices[organizer["name"]] = organizer["index"] # Add index of organizer
        event_attendees.extend([a["name"] for a in attendee_profiles])

        event_profile = {"id": event_id, "name": event_name, "date": str(event_date), "start_time": str(event_start), "end_time": str(event_end), 
                         "location": event_location, "organizer": organizer["name"], "attendees": event_attendees, 
                         "group": event_group}

        for a in event_attendees:
            self.people_calendars[a].append(event_profile)
            self.people[attendee_indices[a]]["calendar"].append(event_profile)
        self.group_calendars[event_group].append(event_profile)
        self.room_calendars[event_location].append(event_profile)
        
        self.events.append(event_profile)
        
    def save_data(self, prefix=""):
        
        with open("people_calendars.json", "w") as file:
            file.write(json.dumps(self.people_calendars))
            
        with open("group_calendars.json", "w") as file:
            file.write(json.dumps(self.group_calendars))
            
        with open("room_calendars.json", "w") as file:
            file.write(json.dumps(self.room_calendars))
            
        
        print("Data Saved")
        
    def save_events(self, prefix=""):
        
        with open("event_calendars.json", "w") as file:
            file.write(json.dumps(self.events))     
        
        print("Events Saved")

        
org = FakeOrg(group_names, rooms, meeting_chunks, n_people=50)
for i in range(50):
    org.create_event()

In [24]:
names = sorted([p["name"] for p in org.people])
for n in names:
    print(n) 

#for p in org.people:
#    print(p["name"])
#print(org.people_calendars)

Abbie Singh
Aimee Williams
Arthur Harrison
Ashley Fowler
Brenda Hayes
Callum Brown
Caroline Perry
Catherine Green
Chloe Stevenson
Clive Howard
Daniel O'Brien
Daniel Smith
Denise Knowles
Diane Gregory
Elaine Vincent
Elliott Harrison
Frank Hunter
Grace Stone
Guy Robinson
Hugh Elliott
Hugh Smith
James Fox
James Jones
Janet Walsh
Jeremy Read
Jodie Mellor
Julian Payne
Katherine Richards
Kathleen Brookes
Lee Williams
Lisa Perry
Mathew Thomas
Owen Farrell
Raymond Khan
Reece Thompson
Ronald Cooper
Rosemary Thomson
Ruth Campbell
Sally Ahmed
Samantha Hunter
Sean Hill
Stacey Morris
Stuart Clarke
Tom Clarke
Tom Skinner
Valerie Thompson
Vanessa Gregory
Victor Cooper
Victoria Holland
William Bates


In [4]:
for e, value in org.group_calendars.items():
    print(e)
    for item in value:
        print(item)

Administration
{'id': 'b51801558f', 'name': 'technologies team meeting', 'date': '2021-11-19', 'start_time': '11:30:00', 'end_time': '12:30:00', 'location': 'Omega', 'organizer': 'Sandra Taylor', 'attendees': ['Sandra Taylor', 'Jean Howard'], 'group': 'Administration'}
{'id': '0b013ba605', 'name': 'ROI project meeting', 'date': '2021-11-24', 'start_time': '15:00:00', 'end_time': '16:00:00', 'location': 'Gamma', 'organizer': 'Jean Howard', 'attendees': ['Jean Howard', 'Debra Hardy', 'Julie Bond'], 'group': 'Administration'}
{'id': 'df8a94a503', 'name': 'partnerships team meeting', 'date': '2021-12-17', 'start_time': '09:00:00', 'end_time': '10:00:00', 'location': 'Sigma', 'organizer': 'Julie Bond', 'attendees': ['Julie Bond', 'Debra Hardy', 'Sandra Taylor'], 'group': 'Administration'}
{'id': '907e09b809', 'name': 'partnerships project meeting', 'date': '2021-12-06', 'start_time': '08:00:00', 'end_time': '09:00:00', 'location': 'Sigma', 'organizer': 'Jean Howard', 'attendees': ['Jean How

In [5]:
for key, value in org.room_calendars.items():
    print(key, value)
org.save_events()
#org.save_data()

Alpha [{'id': 'e8d832b019', 'name': 'ROI seminar', 'date': '2021-11-29', 'start_time': '10:00:00', 'end_time': '11:00:00', 'location': 'Alpha', 'organizer': 'Gerald Marsh', 'attendees': ['Gerald Marsh', 'Gerald Moore'], 'group': 'Statistics'}, {'id': 'cb90fd4cf4', 'name': 'solutions status update', 'date': '2021-11-22', 'start_time': '12:00:00', 'end_time': '13:00:00', 'location': 'Alpha', 'organizer': 'Roy Turner', 'attendees': ['Roy Turner', 'Jack Palmer'], 'group': 'Philosophy'}, {'id': 'e787113511', 'name': 'supply-chains project meeting', 'date': '2021-12-02', 'start_time': '10:30:00', 'end_time': '11:30:00', 'location': 'Alpha', 'organizer': 'Abbie Williams', 'attendees': ['Abbie Williams', 'Keith Watts', 'Adam Tucker'], 'group': 'Mathematics'}, {'id': 'a6fd3472c0', 'name': 'e-tailers discussion', 'date': '2022-01-04', 'start_time': '10:00:00', 'end_time': '11:00:00', 'location': 'Alpha', 'organizer': 'Arthur Middleton', 'attendees': ['Arthur Middleton', 'Thomas Barton', 'Olivia 

In [31]:
with open("people_calendars.json", "r") as file:
    j = json.load(file)
    file.close()

people = [{'label': key, 'value': key} for key, value in j.items()]
print(people)

[{'label': 'Christina Lewis', 'value': 'Christina Lewis'}, {'label': 'John Hoffman', 'value': 'John Hoffman'}, {'label': 'Heather Gillespie', 'value': 'Heather Gillespie'}, {'label': 'Peggy Rubio', 'value': 'Peggy Rubio'}, {'label': 'Zoe Park', 'value': 'Zoe Park'}, {'label': 'Jessica Larson', 'value': 'Jessica Larson'}, {'label': 'Michelle Contreras', 'value': 'Michelle Contreras'}, {'label': 'George Phillips', 'value': 'George Phillips'}, {'label': 'Andrew Castillo', 'value': 'Andrew Castillo'}, {'label': 'Kim Dunn', 'value': 'Kim Dunn'}, {'label': 'Matthew Davis', 'value': 'Matthew Davis'}, {'label': 'Richard Cameron', 'value': 'Richard Cameron'}, {'label': 'Linda Thompson', 'value': 'Linda Thompson'}, {'label': 'Ashley Gentry', 'value': 'Ashley Gentry'}, {'label': 'William Myers', 'value': 'William Myers'}, {'label': 'Ernest Gibson', 'value': 'Ernest Gibson'}, {'label': 'Jennifer Hoffman', 'value': 'Jennifer Hoffman'}, {'label': 'Alexis Perez', 'value': 'Alexis Perez'}, {'label': '